# Find My New City

### Many people wish to move cities and they are unsure on which city they should move to, one natural option to consider is moving to a city that is the most similar to your own. This project will solve that problem, it will take a users city based in the US as an input (such as a university student who has received multiple offers in different cities), after this then the program will cluster and classify cities based on their similarity to each other. After this is complete the city/cities which are most similar to the users original city will be displayed for them as well as the cities level of similarity to the original. 

### To complete the project I will be using US cities data (found here https://simplemaps.com/data/us-cities) and the FourSquare API together to obtain information about the various cities. Data such as venues and venue types will be extracted for each city in the United States. This data will then be used as dimensions to feed a machine learning algorithm which will then classify the cities into different clusters.

### Target audience: Those wishing to move cities due to any number of reasons. One such example is someone who has received multiple university offers in different cities and wishes to find a town that closely resembles their own.

### Data used: https://simplemaps.com/data/us-cities and using the FourSquare API (https://foursquare.com) to extract venue types within a certain radius around the cities.

In [ ]:
import numpy as np
import pandas as pd
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: | 